In [41]:
import tkinter as tk
from tkinter import filedialog

In [42]:
root = tk.Tk()
root.withdraw()

m_path = filedialog.askopenfilename()
m_path

'C:/Users/lci734/OneDrive - AFRY/Documents/01_poyry/01_projetos/03_usiminas/lingotamento/Maconomy/Maconomy lingotamento 23-07-12 - HH.xlsx'

In [43]:
from datetime import datetime

def validar_data(prompt):
    while True:
        data_input = input(prompt)
        try:
            data_status = datetime.strptime(data_input, "%d/%m/%Y").date()
            return data_status
        except ValueError:
            print("Data inválida. Por favor, digite uma data no formato DD/MM/YYYY.")

data_status = validar_data("Digite a data de status (DD/MM/YYYY): ")
print("Data validada:", data_status)

Digite a data de status (DD/MM/YYYY): 12/07/2023
Data validada: 2023-07-12


In [44]:
def validar_projeto(prompt):
    while True:
        projeto_input = input(prompt)
        if len(projeto_input) == 13:
            projeto = projeto_input
            return projeto
        else:
            print('Nº do Projeto inválido. Por favor, digite novamente.')
        
projeto = validar_projeto('Digite o Nº do Projeto (1090XXXXX-XXX): ')
print('Nº do Projeto validado:',projeto )

Digite o Nº do Projeto (1090XXXXX-XXX): 109004000-001
Nº do Projeto validado: 109004000-001


In [45]:
tipo = "MACONOMY"

In [46]:
import pandas as pd

In [47]:
df = pd.read_excel(m_path)
df['category'] = df['Employee Category No.'].str.split('-').str[0]

In [48]:
import sqlite3 as sql

In [49]:
con = sql.connect('teste_00.db')
cur = con.cursor()

In [50]:
# pegar id_activity
cur.execute("SELECT * FROM activities")
activities = cur.fetchall()
activities = pd.DataFrame(activities)
activities.columns = ['id_activity', 'activity', 'indice', 'unidade', 'id_disciplina']
activities.head()

df = df.merge(activities[['activity', 'id_activity']], 
                           left_on='Activity Name', 
                           right_on='activity',
                           how='left')

df.id_activity = df.id_activity.astype('object')

In [51]:
# transformando valores em objetos (strings)
df['Entry No.'] = df['Entry No.'].astype('object')
df['EntryDate'] = df['EntryDate'].dt.date
df['Quantity Reg.'] = df['Quantity Reg.'].astype('object')
df['Base Salary Cost, Currency'] = df['Base Salary Cost, Currency'].astype('object')
df['Cost, Reg.'] = df['Cost, Reg.'].astype('object')

In [52]:
# inserir categorias novas na tabela categorias
categorias_ineditas = []
df_cat = df.loc[(df.category.notnull()) & (df.id_activity.notnull())]
for value in list(zip(df_cat.category.values, df_cat.id_activity.values,)):
    cur.execute("SELECT COUNT(*) FROM categorias WHERE categoria = ? AND id_activity = ?", value)
    count = cur.fetchone()[0]
    if (count == 0) & (value[0] != None) & (value[1] != None): 
        categorias_ineditas.append(value)
        
categorias_ineditas = list(set(categorias_ineditas))
cur.executemany("INSERT INTO categorias (categoria, id_activity) VALUES (?, ?)", categorias_ineditas)   

In [53]:
cur.execute("SELECT * FROM categorias")
categorias = pd.DataFrame(cur.fetchall())
categorias.columns = ['id_categoria', 'categoria', 'id_activity']

df = df.merge(categorias[['categoria', 'id_activity', 'id_categoria']], 
                           left_on=['category','id_activity'], 
                           right_on=['categoria','id_activity'],
                           how='left')

df.id_categoria = df.id_categoria.astype('object')

In [54]:
# inserir projeto na tabela projetos, caso não exista
insert_query = "INSERT OR IGNORE INTO projetos (n_projeto) VALUES (?)"
cur.execute(insert_query, (projeto,))

query = "SELECT * FROM projetos WHERE n_projeto = ?"
cur.execute(query, (projeto,))
id_projeto = cur.fetchall()[0][1]

# inserir registro na tabela registros, caso não exista
check_query = "SELECT COUNT(*) FROM registros WHERE id_projeto = ? AND data_status = ? AND tipo = ?"
cur.execute(check_query, (id_projeto, data_status, tipo,))
unico = cur.fetchall()[0][0]

if unico == 0:
    insert_query = "INSERT OR IGNORE INTO registros (id_projeto, data_status, tipo) VALUES (?, ?, ?)"
    cur.execute(insert_query, (id_projeto, data_status, tipo,))
    print('Registro inserido')
else:
    print('Registro já existente.')
    
# inserir custo na tabela custos, caso não existam
query = "SELECT * FROM registros WHERE id_projeto = ? AND data_status = ? AND tipo = ?"
cur.execute(query, (id_projeto, data_status, tipo,))
id_registro = cur.fetchall()[0][0]

check_query = "SELECT COUNT(*) FROM maconomy WHERE id_registro = ?"
cur.execute(check_query, (id_registro,))
unico = cur.fetchall()[0][0]

if unico == 0:  
    
    entry_no = df['Entry No.'].values
    entry_date = df['EntryDate'].values
    employee_name = df['Employee Name'].values
    id_categoria = df['id_categoria'].values
    task = df['Task'].values
    quantidade = df['Quantity Reg.'].values
    base_cost = df['Base Salary Cost, Currency'].values
    cost_reg = df['Cost, Reg.'].values
    try:
        subprojeto = df['Subprojeto'].values
    except:
        subprojeto = [None] * len(entry_no)
    id_registro = [id_registro] * len(entry_no)
    
    insert_query = """INSERT INTO maconomy 
    (
    id_registro,
    entry_no,
    entry_date,
    employee_name,
    id_categoria,
    task,
    quantidade,
    base_cost,
    cost_reg,
    subprojeto
    )
    VALUES
    (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
    cur.executemany(insert_query, list(zip(id_registro,entry_no,entry_date,employee_name,
                                           id_categoria,task,quantidade,base_cost,cost_reg,
                                           subprojeto)))
    print('MACONOMY inserido')
else:
    print('MACONOMY já existente.')

Registro inserido
MACONOMY inserido


In [55]:
con.commit()
con.close()